### Processing data to run PCA and then NN

In [ ]:
cd ../Python/

In [ ]:
import readPBNData.description as rd
import readPBNData.images as ri

In [ ]:
csvs, fileLike = rd.openZip('../Data/train_info.csv.zip')
lines, head = rd.readCSV(fileLike[0])

In [ ]:
print(head)
lines[:2]

In [ ]:
cols = rd.columns(lines,head)
cols.keys()

In [ ]:
colsdf=pd.DataFrame(cols)
colsdf

In [ ]:
cols['title'][:10]

In [ ]:
img = ri.openZipImage('../Data/train_1.zip',cols['filename'][0],prefix='train_1')

In [ ]:
img.show()

In [ ]:
new2= img.resize(size=(100,100))

In [ ]:
print(new2.getdata())

In [ ]:
new= img.resize(size=(100,100))

In [ ]:
new

In [ ]:
print(new.format, new.size, new.mode)

In [ ]:
#r, g, b = new.split()
#r

In [ ]:
import zipfile
import os
zf = zipfile.ZipFile('../Data/train_1.zip','r')
filenames = zf.namelist()[1:]
filenames[:10]
#print([os.path.basename(h) for h in filenames])

#os.path.basename(filenames[2])

In [ ]:
len(filenames)

In [ ]:
import readPBNData.description as rd
import readPBNData.images as ri
from PIL import Image, ImageMath
import numpy as np

mylist=[]
invalid = pd.DataFrame()
for h in range(2000):
    #del img, new, cleanlist
    img = ri.openZipImage('../Data/train_1.zip',filenames[h])
    try:
        new= img.resize(size=(100,100))
    except IOError as e:
        invalid = invalid.append([[filenames[h], e]],ignore_index=True)
    cleanlist=np.array(new).flatten()
    mylist.append([cleanlist,os.path.basename(filenames[h])])
mylist=np.array(mylist)


In [ ]:
invalid

In [ ]:
mylist[1200:1300]

In [ ]:
#mylist2[:,0][0]
df = pd.DataFrame({'pixels':mylist[:,0],'filename':mylist[:,1]})
#mylist[:,1]

In [ ]:
df

In [ ]:
result = pd.merge(df, colsdf, on='filename')
#result = pd.concat([df, colsdf], axis=1, join_axes=[df.filename])
result

In [ ]:
grouped = result.groupby('artist')
np.sort(grouped.size())[-200:]
#np.sort(grouped.size())[-50:]
#grouped.size().sort_values(axis=0, ascending=True)
#result.groupby('artist', sort=True).sum()

In [ ]:
#grouped.groups
final=grouped.filter(lambda x: len(x) > 9)
final
#result.artist[np.sort(grouped.size())[-100:]]

In [ ]:
# Create classes based on artist (only 24 artist in this sample, the most frequent ones)
uniq = sorted(list(set(final['artist'])))
artnum = [uniq.index(x) for x in final['artist']]
# Put the data in the right format to run the NN
frame2=[pd.DataFrame(final['pixels'].tolist()),pd.DataFrame(artnum)]
datafinal=pd.concat(frame2, axis=1).dropna(axis=0)
datafinal.columns = [list(range(30001))]

y_class = datafinal[30000]
X_train = datafinal.ix[:,0:29999]
y_train = y_class
#X_test = dtm[n_break:n_final]
#y_test = y_class[n_break:n_final]
print(X_train.shape,y_train.shape,type(y_train))

In [ ]:
#final['pixels'].tolist()
#pd.DataFrame(data=final['pixels'].tolist())
#final['artist']
#X_train
#pd.isnull(X_train[29990])
#X_train.dropna(axis=0).shape
#datafinal=pd.DataFrame({final['pixels'].tolist(),artnum})
#frame2=[pd.DataFrame(final['pixels'].tolist()),pd.DataFrame(artnum)]
#datafinal=pd.concat(frame2, axis=1).dropna(axis=0)
#datafinal.columns = [list(range(30001))]

In [ ]:
#pd.DataFrame(final['pixels'].tolist())
#pd.DataFrame(artnum).transpose()
#datafinal[30000]
#range(10)
#X_train

### Principal Component Analysis to MOvie data

In [ ]:
# Principal Component Analysis to MOvie data

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, RandomizedPCA
from sklearn.preprocessing import scale
%matplotlib inline
# Note that I use scale and then PCA with Whitening, because whitening eliminates
# correlation among features but scaling only affects the features independently
components= 270
pca = PCA( n_components = components,whiten=True )
#pca = RandomizedPCA(n_components = 200, random_state=0 )
X_pca = pca.fit_transform(scale(X_train))
#X_pca_test = pca.transform(scale(X_test))

# Percentage of variance explained for each components
print('explained variance ratio (first n components): %s'
      % str(sum(pca.explained_variance_ratio_)))
print(X_pca.shape)

#eigenvalues = pca.explained_variance_
plt.figure(figsize=(4, 4))
eigencumul=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
plt.plot(range(components),eigencumul)

plt.ylim(0, 110)
plt.xlim(0, components)
#plt.ylabel('Error')
plt.xlabel('# of components')
plt.title('Image Dataset \n Cumulative percentage of variance \n explained by components \n', fontsize=12, ha='center')

### Applying a NN using Theano

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import matplotlib
import theano
import theano.tensor as T
#import pydot_ng
#import pydot
from IPython.display import Image
from IPython.display import SVG
from sklearn.preprocessing import scale
import timeit
from time import time

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)



# Note to TAs: you have to un comment the paragrah below to calculate results for
# the different variables

##Movie data

## Original data
train_y = y_train.astype(np.int32)
#test_y = y_test.astype(np.int32)
train_X = scale(X_train).astype(np.float32)
#test_X = scale(X_test).astype(np.float32)



# Size definitions
num_examples = len(train_X) # training set size
nn_input_dim = train_X.shape[1] # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality (2 for Movie data and 4 for Wine data)
nn_hdim = 3 # hiden layer dimensionality

# Gradient descent parameters 
epsilon = 0.01 # learning rate for gradient descent
reg_lambda = 0.01 # regularization strength 

# Our data vectors
X = T.matrix('X') # matrix of doubles
y = T.lvector('y') # vector of int64

# Shared variables with initial values. We need to learn these.
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim), name='W1')
b1 = theano.shared(np.zeros(nn_hdim), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim), name='b2')

# Forward propagation
# Note: We are just defining the expressions, nothing is evaluated here!
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2) # output probabilties

# The regularization term (optional)
loss_reg = 1./num_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2))) 
# the loss function we want to optimize
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg

# Returns a class prediction
prediction = T.argmax(y_hat, axis=1)

# Theano functions that can be called from our Python code
forward_prop = theano.function([X], y_hat)
calculate_loss = theano.function([X, y], loss)
predict = theano.function([X], prediction)

# Easy: Let Theano calculate the derivatives for us!
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

gradient_step = theano.function(
    [X, y],
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

In [ ]:
# This function learns parameters for the neural network and returns the model.
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 100 iterations
train_errorsNN = []
test_errorsNN = []
num_passes= 2000
def build_model(num_passes=num_passes, print_loss=False):
    
    # Re-Initialize the parameters to random values. We need to learn these.
    # (Needed in case we call this function multiple times)
    np.random.seed(0)
    W1.set_value(np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim))
    b1.set_value(np.zeros(nn_hdim))
    W2.set_value(np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim))
    b2.set_value(np.zeros(nn_output_dim))
    
    # Gradient descent. For each batch...
    for i in range(0, num_passes):
        # This will update our parameters W2, b2, W1 and b1!
        gradient_step(train_X, train_y)
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 100 == 0:
            #print("Loss after iteration %i: %f" %(i, calculate_loss(train_X, train_y)))
            train_errorsNN.append(1. - np.mean(predict(train_X)==train_y))
            #test_errorsNN.append(1. - np.mean(predict(test_X)==test_y))
  
# Build a model with a n-dimensional hidden layer

t0 = time()
build_model(print_loss=True)
print("done in %0.3fs." % (time() - t0))

# Plot the decision boundary
#plot_decision_boundary(lambda x: predict(x))
#plt.title("Decision Boundary for hidden layer size 3")

### Just trying stuff

In [ ]:
# Convert image to array , and then from array to image
im = np.array(new)
#im.flatten().reshape(100,100,3)
im3 = Image.fromarray(im.flatten().reshape(100,100,3))
im3

In [ ]:
im = array(new)
im.shape

In [ ]:
immatrix
imlist=[new, new]
len(imlist)

In [ ]:
imlist=[array(new).reshape(-1, 3), array(new).reshape(-1, 3)]
len(imlist)

In [ ]:
#im.reshape(-1, 3).shape

imlist=[array(new).reshape(-1, 3), array(new).reshape(-1, 3)]
im = imlist[0] # open one image to get size
m,n = im.shape[0:2] # get the size of the images
imnbr = len(imlist) # get the number of images

# create matrix to store all flattened images
#immatrix = array([array(Image.open(im)).flatten()
              #for j in imlist],'f')

# create matrix to store all flattened images
immatrix = array([imlist[i].flatten()
              for i in range(len(imlist))],'f')



In [ ]:
imshow(immean.reshape(m,n,r))

In [ ]:
immatrix.mean(axis=0).shape

In [ ]:
pca2(immatrix)[2]

In [ ]:
V[1].reshape(m,n,r).shape

In [ ]:
immatrix.shape

In [ ]:
V.shape

In [ ]:
mean_X = X.mean(axis=0)
mean_X

In [ ]:
immatrix

In [ ]:
S

In [ ]:
X=immatrix
M = dot(X,X.T)
linalg.eigh(M)

In [ ]:
X.shape